In [1]:
from pathlib import Path
initial_path= os.getcwd()


In [2]:
os.chdir(Path(initial_path).parent)
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from test import *
import sys
# sys.path.append('..')
from src.data.cp_dataset import CPDataset, CPDataLoader
from tensorboardX import SummaryWriter
from src.models.networks import GMM, UnetGenerator,load_checkpoint
#bring the working directory to the top
import time
# os.chdir("..")


In [4]:
def get_opt():
    parser = argparse.ArgumentParser()

    parser.add_argument("--name", default="GMM")
    # parser.add_argument("--name", default="TOM")

    parser.add_argument("--gpu_ids", default="1")
    parser.add_argument('-j', '--workers', type=int, default=1)
    parser.add_argument('-b', '--batch-size', type=int, default=4)

    parser.add_argument("--dataroot", default="data")

    parser.add_argument("--datamode", default="test", choices=["test", "wild"])

    parser.add_argument("--stage", default="GMM")
    # parser.add_argument("--stage", default="TOM")

    parser.add_argument("--data_list", default="test_pairs.txt")
    # parser.add_argument("--data_list", default="test_pairs_same.txt")

    parser.add_argument("--fine_width", type=int, default=192)
    parser.add_argument("--fine_height", type=int, default=256)
    parser.add_argument("--radius", type=int, default=5)
    parser.add_argument("--grid_size", type=int, default=5)

    parser.add_argument('--tensorboard_dir', type=str, default='tensortest', help='save tensorboard infos')

    parser.add_argument('--result_dir', type=str,default='result', help='save result infos')

    parser.add_argument('--checkpoint_GMM', type=str, default='checkpoints/GMM/gmm_final.pth', help='model checkpoint for test')
    parser.add_argument('--checkpoint_TOM', type=str, default='checkpoints/TOM/tom_final.pth', help='model checkpoint for test')

    parser.add_argument("--display_count", type=int, default=1)
    parser.add_argument("--shuffle", action='store_true',default=True,
                        help='shuffle input data')

    opt = parser.parse_args(args=[])
    return opt

In [5]:
opt = get_opt()

In [6]:
print("Start to test stage: %s, named: %s!" % (opt.stage, opt.name))
test_dataset = CPDataset(opt)

Start to test stage: GMM, named: GMM!


In [7]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=opt.batch_size, shuffle=opt.shuffle, num_workers=opt.workers)

In [8]:
import os
if not os.path.exists(opt.tensorboard_dir):
        os.makedirs(opt.tensorboard_dir)
board = SummaryWriter(logdir=os.path.join(opt.tensorboard_dir, opt.name))

In [9]:
model_gmm = GMM(opt)
model_gmm.cuda()
model_gmm.eval()
load_checkpoint(model_gmm, opt.checkpoint_GMM)



initialization method [normal]
initialization method [normal]


MAKE predictions 

In [10]:
from PIL import Image
def save_images(img_tensors, img_names, save_dir):
    for img_tensor, img_name in zip(img_tensors, img_names):
        tensor = (img_tensor.clone()+1)*0.5 * 255
        tensor = tensor.cpu().clamp(0, 255)

        array = tensor.numpy().astype('uint8')
        if array.shape[0] == 1:
            array = array.squeeze(0)
        elif array.shape[0] == 3:
            array = array.swapaxes(0, 1).swapaxes(1, 2)

        Image.fromarray(array).save(os.path.join(save_dir, img_name))

In [11]:
base_name = os.path.basename(opt.checkpoint_GMM)
name = opt.name
save_dir = os.path.join(opt.result_dir, name, opt.datamode)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
warp_cloth_dir = os.path.join(save_dir, 'warp-cloth')
if not os.path.exists(warp_cloth_dir):
    os.makedirs(warp_cloth_dir)
warp_mask_dir = os.path.join(save_dir, 'warp-mask')
if not os.path.exists(warp_mask_dir):
    os.makedirs(warp_mask_dir)
result_dir1 = os.path.join(save_dir, 'result_dir')
if not os.path.exists(result_dir1):
    os.makedirs(result_dir1)
overlayed_TPS_dir = os.path.join(save_dir, 'overlayed_TPS')
if not os.path.exists(overlayed_TPS_dir):
    os.makedirs(overlayed_TPS_dir)
warped_grid_dir = os.path.join(save_dir, 'warped_grid')
if not os.path.exists(warped_grid_dir):
    os.makedirs(warped_grid_dir)

In [12]:
with torch.no_grad():
    for step, inputs in enumerate(test_loader):
        iter_start_time = time.time()
       

        c_names = inputs['c_name']
        im_names = inputs['im_name']
        im = inputs['image'].cuda()
        im_pose = inputs['pose_image'].cuda()
        im_h = inputs['head'].cuda()
        shape = inputs['shape'].cuda()
        agnostic = inputs['agnostic'].cuda()
        c = inputs['cloth'].cuda()
        cm = inputs['cloth_mask'].cuda()
        im_c = inputs['parse_cloth'].cuda()
        im_g = inputs['grid_image'].cuda()
        shape_ori = inputs['shape_ori']  # original body shape without blurring

        grid, theta = model_gmm(agnostic, cm)
        warped_cloth = F.grid_sample(c, grid, padding_mode='border')
        warped_mask = F.grid_sample(cm, grid, padding_mode='zeros')
        warped_grid = F.grid_sample(im_g, grid, padding_mode='zeros')
        overlay = 0.7 * warped_cloth + 0.3 * im

        visuals = [[im_h, shape, im_pose],
                    [c, warped_cloth, im_c],
                    [warped_grid, (warped_cloth+im)*0.5, im]]

        # save_images(warped_cloth, c_names, warp_cloth_dir)
        # save_images(warped_mask*2-1, c_names, warp_mask_dir)
        save_images(warped_cloth, im_names, warp_cloth_dir)
        save_images(warped_mask * 2 - 1, im_names, warp_mask_dir)
        save_images(shape_ori.cuda() * 0.2 + warped_cloth *
                    0.8, im_names, result_dir1)
        save_images(warped_grid, im_names, warped_grid_dir)
        save_images(overlay, im_names, overlayed_TPS_dir)

        if (step+1) % opt.display_count == 0:
            # board_add_images(board, 'combine', visuals, step+1)
            t = time.time() - iter_start_time
            print('step: %8d, time: %.3f' % (step+1, t), flush=True)

step:        1, time: 2.925
step:        2, time: 0.109


c:\Users\luca9\miniconda3\envs\ml\lib\site-packages\torch\nn\functional.py:4194: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "


step:        3, time: 0.120
step:        4, time: 0.146
step:        5, time: 0.119
step:        6, time: 0.119
step:        7, time: 0.115
step:        8, time: 0.120
step:        9, time: 0.119
step:       10, time: 0.129
step:       11, time: 0.119
step:       12, time: 0.118
step:       13, time: 0.114
step:       14, time: 0.155
step:       15, time: 0.119
step:       16, time: 0.126
step:       17, time: 0.130
step:       18, time: 0.121
step:       19, time: 0.127
step:       20, time: 0.121
step:       21, time: 0.141
step:       22, time: 0.135
step:       23, time: 0.126
step:       24, time: 0.119
step:       25, time: 0.172
step:       26, time: 0.162
step:       27, time: 0.147
step:       28, time: 0.110
step:       29, time: 0.109
step:       30, time: 0.105
step:       31, time: 0.114
step:       32, time: 0.104
step:       33, time: 0.105
step:       34, time: 0.096
step:       35, time: 0.101
step:       36, time: 0.100
step:       37, time: 0.100
step:       38, time

In [13]:
with torch.no_grad():
    for step, inputs in enumerate(test_loader):
        
        iter_start_time = time.time()
       

        c_names = inputs['c_name']
        im_names = inputs['im_name']
        im = inputs['image'].cuda()
        im_pose = inputs['pose_image'].cuda()
        im_h = inputs['head'].cuda()
        shape = inputs['shape'].cuda()
        agnostic = inputs['agnostic'].cuda()
        c = inputs['cloth'].cuda()
        cm = inputs['cloth_mask'].cuda()
        im_c = inputs['parse_cloth'].cuda()
        im_g = inputs['grid_image'].cuda()
        shape_ori = inputs['shape_ori']  # original body shape without blurring

        grid, theta = model_gmm(agnostic, cm)
        warped_cloth = F.grid_sample(c, grid, padding_mode='border')
        warped_mask = F.grid_sample(cm, grid, padding_mode='zeros')
        warped_grid = F.grid_sample(im_g, grid, padding_mode='zeros')
        overlay = 0.7 * warped_cloth + 0.3 * im

        visuals = [[im_h, shape, im_pose],
                    [c, warped_cloth, im_c],
                    [warped_grid, (warped_cloth+im)*0.5, im]]

        save_images(warped_cloth, im_names, warp_cloth_dir)
        save_images(warped_mask * 2 - 1, im_names, warp_mask_dir)
        save_images(shape_ori.cuda() * 0.2 + warped_cloth *
                    0.8, im_names, result_dir1)
        save_images(warped_grid, im_names, warped_grid_dir)
        save_images(overlay, im_names, overlayed_TPS_dir)

        if (step+1) % opt.display_count == 0:
            t = time.time() - iter_start_time
            print('step: %8d, time: %.3f' % (step+1, t), flush=True)

        
        # opt.stage="TOM"
        # for idx,cloth in enumerate(c_names):
        #     print(cloth)
        #     cv2.imshow(cloth, c.detach().cpu().numpy()[idx].transpose(1, 2, 0))
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()

        # outputs = model_tom(torch.cat([agnostic, c, cm], 1))
        # p_rendered, m_composite = torch.split(outputs, 3, 1)
        # p_rendered = F.tanh(p_rendered)
        # m_composite = F.sigmoid(m_composite)
        # p_tryon = c * m_composite + p_rendered * (1 - m_composite)

        # for idx,cloth in enumerate(c_names):
        #     print(cloth)
        #     cv2.imshow(cloth, p_rendered.detach().cpu().numpy()[idx].transpose(1, 2, 0))
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()
        # break




step:        1, time: 0.147
step:        2, time: 0.138
step:        3, time: 0.140
step:        4, time: 0.113
step:        5, time: 0.090
step:        6, time: 0.086
step:        7, time: 0.087
step:        8, time: 0.087
step:        9, time: 0.083
step:       10, time: 0.087
step:       11, time: 0.088
step:       12, time: 0.093
step:       13, time: 0.098
step:       14, time: 0.087
step:       15, time: 0.086
step:       16, time: 0.086
step:       17, time: 0.084
step:       18, time: 0.094
step:       19, time: 0.094
step:       20, time: 0.087
step:       21, time: 0.083
step:       22, time: 0.083
step:       23, time: 0.090
step:       24, time: 0.090
step:       25, time: 0.094
step:       26, time: 0.091
step:       27, time: 0.093
step:       28, time: 0.089
step:       29, time: 0.087
step:       30, time: 0.087
step:       31, time: 0.086
step:       32, time: 0.089
step:       33, time: 0.090
step:       34, time: 0.090
step:       35, time: 0.089
step:       36, time

In [14]:
target = "data\\test"
# target = target.replace("\\","/")

In [15]:
import shutil

if os.path.exists(os.path.join(target, "warp-cloth")):
    shutil.rmtree(os.path.join(target, "warp-cloth"))
if os.path.exists(os.path.join(target, "warp-mask")):
    shutil.rmtree(os.path.join(target, "warp-mask"))
# os.path.join("result/GMM/test/warp-cloth", target)

print("warp-mask moved:",shutil.move("result/GMM/test/warp-mask", target))
print("warp-cloth moved:",shutil.move("result/GMM/test/warp-cloth", target))


warp-mask moved: data\test\warp-mask
warp-cloth moved: data\test\warp-cloth


In [16]:
def get_opt():
    parser = argparse.ArgumentParser()

    # parser.add_argument("--name", default="GMM")
    parser.add_argument("--name", default="TOM")

    parser.add_argument("--gpu_ids", default="1")
    parser.add_argument('-j', '--workers', type=int, default=1)
    parser.add_argument('-b', '--batch-size', type=int, default=4)

    parser.add_argument("--dataroot", default="data")

    # parser.add_argument("--datamode", default="train")
    parser.add_argument("--datamode", default="test")
    # parser.add_argument("--datamode", default="wild")

    # parser.add_argument("--stage", default="GMM")
    parser.add_argument("--stage", default="TOM")

    # parser.add_argument("--data_list", default="train_pairs.txt")
    parser.add_argument("--data_list", default="test_pairs.txt")
    # parser.add_argument("--data_list", default="wild_pairs.txt")
    # parser.add_argument("--data_list", default="test_pairs_same.txt")

    parser.add_argument("--fine_width", type=int, default=192)
    parser.add_argument("--fine_height", type=int, default=256)
    parser.add_argument("--radius", type=int, default=5)
    parser.add_argument("--grid_size", type=int, default=5)

    parser.add_argument('--tensorboard_dir', type=str,
                        default='tensorboard', help='save tensorboard infos')

    parser.add_argument('--result_dir', type=str,
                        default='result', help='save result infos')

    # parser.add_argument('--checkpoint', type=str, default='checkpoints/GMM/gmm_final.pth', help='model checkpoint for test')
    parser.add_argument('--checkpoint', type=str, default='checkpoints/TOM/tom_final.pth', help='model checkpoint for test')

    parser.add_argument("--display_count", type=int, default=1)
    parser.add_argument("--shuffle", action='store_true',default=True,
                        help='shuffle input data')

    opt = parser.parse_args(args=[])
    return opt

In [17]:
opt = get_opt()
test_dataset = CPDataset(opt)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=opt.batch_size, shuffle=opt.shuffle, num_workers=opt.workers)

In [18]:
base_name = os.path.basename(opt.checkpoint)
# save_dir = os.path.join(opt.result_dir, base_name, opt.datamode)
save_dir = os.path.join(opt.result_dir, opt.name, opt.datamode)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
try_on_dir = os.path.join(save_dir, 'try-on')
if not os.path.exists(try_on_dir):
    os.makedirs(try_on_dir)
p_rendered_dir = os.path.join(save_dir, 'p_rendered')
if not os.path.exists(p_rendered_dir):
    os.makedirs(p_rendered_dir)
m_composite_dir = os.path.join(save_dir, 'm_composite')
if not os.path.exists(m_composite_dir):
    os.makedirs(m_composite_dir)
im_pose_dir = os.path.join(save_dir, 'im_pose')
if not os.path.exists(im_pose_dir):
    os.makedirs(im_pose_dir)
shape_dir = os.path.join(save_dir, 'shape')
if not os.path.exists(shape_dir):
    os.makedirs(shape_dir)
im_h_dir = os.path.join(save_dir, 'im_h')
if not os.path.exists(im_h_dir):
    os.makedirs(im_h_dir)  # for test data

In [19]:
model_tom = UnetGenerator(26, 4, 6, ngf=64, norm_layer=nn.InstanceNorm2d)  # CP-VTON+
model_tom.cuda()
model_tom.eval()
checkpoint_tom= "checkpoints/TOM/tom_final.pth"
load_checkpoint(model_tom, checkpoint_tom)

In [20]:
with torch.no_grad():
    print('Dataset size: %05d!' % (len(test_loader.dataset)), flush=True)
    for step, inputs in enumerate(test_loader):
        iter_start_time = time.time()

        im_names = inputs['im_name']
        im = inputs['image'].cuda()
        im_pose = inputs['pose_image']
        im_h = inputs['head']
        shape = inputs['shape']

        agnostic = inputs['agnostic'].cuda()
        c = inputs['cloth'].cuda()
        cm = inputs['cloth_mask'].cuda()

        # outputs = model(torch.cat([agnostic, c], 1))  # CP-VTON
        outputs = model_tom(torch.cat([agnostic, c, cm], 1))  # CP-VTON+
        p_rendered, m_composite = torch.split(outputs, 3, 1)
        p_rendered = F.tanh(p_rendered)
        m_composite = F.sigmoid(m_composite)
        p_tryon = c * m_composite + p_rendered * (1 - m_composite)

        visuals = [[im_h, shape, im_pose],
                   [c, 2*cm-1, m_composite],
                   [p_rendered, p_tryon, im]]

        
        # cv2.imshow("try on", p_tryon[0].detach().cpu().numpy().transpose(1, 2, 0))
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        save_images(p_tryon, im_names, try_on_dir)
        save_images(im_h, im_names, im_h_dir)
        save_images(shape, im_names, shape_dir)
        save_images(im_pose, im_names, im_pose_dir)
        save_images(m_composite, im_names, m_composite_dir)
        save_images(p_rendered, im_names, p_rendered_dir)  # For test data

        if (step+1) % opt.display_count == 0:
            t = time.time() - iter_start_time
            print('step: %8d, time: %.3f' % (step+1, t), flush=True)

Dataset size: 02032!


c:\Users\luca9\miniconda3\envs\ml\lib\site-packages\torch\nn\functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
c:\Users\luca9\miniconda3\envs\ml\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


step:        1, time: 0.295
step:        2, time: 0.118
step:        3, time: 0.116
step:        4, time: 0.112
step:        5, time: 0.123
step:        6, time: 0.107
step:        7, time: 0.117
step:        8, time: 0.111
step:        9, time: 0.109
step:       10, time: 0.103
step:       11, time: 0.103
step:       12, time: 0.108
step:       13, time: 0.109
step:       14, time: 0.117
step:       15, time: 0.115
step:       16, time: 0.104
step:       17, time: 0.112
step:       18, time: 0.104
step:       19, time: 0.099
step:       20, time: 0.101
step:       21, time: 0.104
step:       22, time: 0.104
step:       23, time: 0.102
step:       24, time: 0.115
step:       25, time: 0.104
step:       26, time: 0.103
step:       27, time: 0.104
step:       28, time: 0.098
step:       29, time: 0.105
step:       30, time: 0.100
step:       31, time: 0.111
step:       32, time: 0.108
step:       33, time: 0.103
step:       34, time: 0.109
step:       35, time: 0.107
step:       36, time

In [21]:
import cv2

In [22]:
import matplotlib.pyplot as plt
import numpy as np

Calculate SSIM - optional visualizations

In [23]:
import os
from skimage.metrics import structural_similarity
import lpips
from torchvision.transforms import ToTensor, Lambda, transforms
transform = transforms.Compose([ToTensor(),transforms.Normalize( mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])

loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
target_dir = 'result/TOM/test/try-on'
initial_dir = 'data/test/image'

count = 0
tot_ssim_score = 0
tot_lpips= 0
for filename in os.listdir(target_dir):
    imageA = cv2.imread(target_dir + "/" + filename)
    imageB = cv2.imread(initial_dir + "/" + filename)

    
    # 4. Convert the images to grayscale
    grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    
    # numpy_horizontal = np.hstack((imageB , imageA))
    # cv2.imshow('Numpy Horizontal', numpy_horizontal)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # 5. Compute the Structural Similarity Index (SSIM) between the two
    #    images, ensuring that the difference image is returned
    (score, diff) = structural_similarity(grayA, grayB, full=True)
    diff = (diff * 255).astype("uint8")
    count += 1
    tot_ssim_score += score
    #print(score)
    # 6. calculate LPIPS
    
    #use transforms for the normalization
    img0 = transform(imageA) # image should be RGB, IMPORTANT: normalized to [-1,1]
    img1 = transform(imageB)
    d = loss_fn_alex(img0, img1)
    tot_lpips += d

ssim_score = tot_ssim_score / count
lpips_score = tot_lpips / count
# 6. You can print only the score if you want
print("SSIM: {}".format(ssim_score))
print("LPIPS: {}".format(lpips_score.item()))

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\luca9\miniconda3\envs\ml\lib\site-packages\lpips\weights\v0.1\alex.pth
SSIM: 0.7434842484185319
LPIPS: tensor([[[[0.2678]]]], grad_fn=<DivBackward0>)


0.2677542269229889

In [ ]:
img1 = cv2.imread("lena.jpg")
w1 = img1.shape[1]

img2 = cv2.imread("barn.jpg")
w2 = img2.shape[1]

img3 = cv2.imread("monet2.jpg")
w3 = img3.shape[1]

# get maximum width
ww = max(w1, w2, w3)

# pad images with transparency in width
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2BGRA)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2BGRA)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2BGRA)
img1 = cv2.copyMakeBorder(img1, 0, 0, 0, ww-w1, borderType=cv2.BORDER_CONSTANT, value=(0,0,0,0))
img2 = cv2.copyMakeBorder(img2, 0, 0, 0, ww-w2, borderType=cv2.BORDER_CONSTANT, value=(0,0,0,0))
img3 = cv2.copyMakeBorder(img3, 0, 0, 0, ww-w3, borderType=cv2.BORDER_CONSTANT, value=(0,0,0,0))

# stack images vertically
result = cv2.vconcat([img1, img2, img3])

# write result to disk
cv2.imwrite("image_stack.png", result)

cv2.imshow("RESULT", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

SHOW DIFFERENCE BETWEEN PREDICTIONS AND GROUNDTRUTH WITH DIFF

In [ ]:
img1 = imageA
w1 = img1.shape[1]
img2 = imageB
w2 = img2.shape[1]
img3 = diff
w3 = img3.shape[1]

# get maximum width
ww = max(w1, w2, w3)

# pad images with transparency in width
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2BGRA)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2BGRA)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2BGRA)
img1 = cv2.copyMakeBorder(img1, 0, 0, 0, ww-w1, borderType=cv2.BORDER_CONSTANT, value=(0,0,0,0))
img2 = cv2.copyMakeBorder(img2, 0, 0, 0, ww-w2, borderType=cv2.BORDER_CONSTANT, value=(0,0,0,0))
img3 = cv2.copyMakeBorder(img3, 0, 0, 0, ww-w3, borderType=cv2.BORDER_CONSTANT, value=(0,0,0,0))

# stack images vertically
result = cv2.vconcat([img1, img2, img3])

# write result to disk
cv2.imwrite("image_stack.png", result)

cv2.imshow("RESULT", result)
cv2.waitKey(0)
cv2.destroyAllWindows()